## Resource-Constrained Project Scheduling Problem with Transfer Times

This notebook demonstrates how to model and solve the Resource-Constrained Project Scheduling Problem with Sequence-Dependent Setup Times
using Constraint Programming with IBM’s CP Optimizer via the [`docplex.cp`](https://ibmdecisionoptimization.github.io/docplex-doc/cp/refman.html) Python API.
This problem extends the classical RCPSP (see [`rcpsp.ipynb`](https://github.com/radovluk/CP_Cookbook/blob/main/notebooks/rcpsp.ipynb)). [Problem defintion from Vilem Heinz](https://www.overleaf.com/1895916189bqhxjfkpwvws#151a3f)

### Problem Definiton

The Resource-Constrained Project Scheduling Problem with Trasfer Times (RCPSPTT) is an extension of the RCPSP problem defined. In RCPSP, activities are executed using available resources, precedence relationships between them must be adhered to, and the total makespan is minimized. Compared to the RCPSP, RCPSPTT introduces transfer times between activities. This means that resources do not have to be directly available after the activity that uses them finishes; rather, a transfer of resources to the following activity might be required. The transfer time depends on both the activities and the resource transferred.

Formally, the RCPSPTT problem is defined as follows:

Let $\mathcal{A}=\{0,1,\ldots,\hat{A}\}$, $\ \hat{A}\in\mathbb{N}$, be a set of activities with indices $i,j\in\mathcal{A}$.

Let $P\in\mathbb{N}^{|\mathcal{A}|}$ be a vector of activity processing times.

Let $\mathcal{E}=\{(i,j)\mid \text{activity } i \text{ precedes activity } j\}$ be a set of precedences.

Let $\mathcal{R}=\{0,1,\ldots,\hat{R}\}$, $\ \hat{R}\in\mathbb{N}$, be a set of primary resources indexed by $r\in\mathcal{R}$.

Let $C\in\mathbb{N}^{|\mathcal{R}|}$ be a vector of primary resource capacities.

Let $\mathbf{Q}\in\mathbb{Z}_{{\ge 0}}^{|\mathcal{A}|\times|\mathcal{R}|}$ be the activity–resource consumption matrix, with entries $\mathbf{Q}_{i,r}$ denoting the consumption of resource $r$ by activity $i$ ($i=0 \Rightarrow \mathbf{Q}_{i,r}=C_r$).

Let $\Delta\in\mathbb{Z}_{\ge 0}^{|\mathcal{A}|\times|\mathcal{A}|\times|\mathcal{R}|}$ be a matrix of transfer delays, with entries $\Delta_{i,j,r}$ denoting the delay required if resource $r$ is transferred between activities $i$ and $j$.

### CPLEX Formulation

$$
\begin{aligned}
\min \quad
& \operatorname{end}(a_{|\mathcal{A}|-1})
\qquad &\qquad & \text{(1)} \\[2mm]
\text{s.t.} \quad
& \operatorname{endBeforeStart}(a_i, a_j),
\qquad & \forall (i, j) \in \mathcal{E}
\quad & \text{(2)} \\[1mm]
& \sum_{j:(0, j, r) \in \mathcal{T}} f_{0, j, r} = C_r,
\qquad & \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(3)} \\[1mm]
& f_{i,j,r} \ge 1 \implies \operatorname{presenceOf}(z_{i,j,r}) = 1,
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r} = 0
\quad & \text{(4)} \\[2mm]
& \sum_{\substack{j, k:(j, i, k) \in \mathcal{T}, \\ \Delta_{j, i, k} > 0, k=r}} \operatorname{heightAtStart}(z_{j, i, k}, \operatorname{pulse}(z_{j, i, k}, (0, U_{j, i, k}))) + \sum_{\substack{j, k:(j, i, k) \in \mathcal{T}, \\ \Delta_{j, i, k} = 0, k=r}} f_{j, i, k} = Q_{i, r},
\qquad & \forall i \in \{1, \dots, |\mathcal{A}| - 1\}, \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(5)} \\[2mm]
& \sum_{\substack{j, k:(i, j, k) \in \mathcal{T}, \\ \Delta_{i, j, k} > 0, k=r}} \operatorname{heightAtStart}(z_{i, j, r}, \operatorname{pulse}(z_{i, j, r}, (0, U_{i, j, r}))) + \sum_{\substack{j, k:(i, j, k) \in \mathcal{T}, \\ \Delta_{i, j, k} = 0, k=r}} f_{i, j, k} = Q_{i, r},
\qquad & \forall i \in \{0, \dots, |\mathcal{A}| - 2\}, \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(6)} \\[2mm]
& \operatorname{endBeforeStart}(a_i, z_{i, j, r}),
\qquad & \forall (i, j, r) \in \mathcal{T}
\quad & \text{(7)} \\[1mm]
& \operatorname{endBeforeStart}(z_{i, j, r}, a_j),
\qquad & \forall (i, j, r) \in \mathcal{T}
\quad & \\[2mm]
& \sum_{i: \mathbf{Q}_{i, r} > 0} \operatorname{pulse}(a_i, \mathbf{Q}_{i, r}) + \sum_{\substack{(i, j, k) \in \mathcal{T}, \\ \Delta_{i, j, k} > 0, k=r}} \operatorname{pulse}(z_{i, j, r}, (0, U_{i, j, r})) \le C_r,
\qquad & \forall r \in \{0, \dots, |\mathcal{R}| - 1\}
\quad & \text{(8)} \\[2mm]
& a_i: \text{mandatory interval var, length } p_i,
\qquad & \forall i \in \mathcal{A}
\quad & \text{(9a)} \\[1mm]
& f_{i,j,r} \in \mathbb{Z}, f_{i,j,r} \in [0, U_{i,j,r}],
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r}=0
\quad & \text{(9b)} \\[1mm]
& z_{i,j,r}: \text{optional interval var, length } \Delta_{i,j,r},
\qquad & \forall (i, j, r) \in \mathcal{T}, \Delta_{i,j,r}>=0
\quad & \text{(9c)}
\end{aligned}
$$

**Objective:**
* (1) Minimize the makespan (the end time of the final sink activity $a_{|\mathcal{A}|-1}$).


**Modeling constraints:**
* (2) Enforces precedence relations $\mathcal{E}$ between activities, ensuring an activity must finish before its successor can start.
* (3) Ensures the total resource flow $r$ from the source node (activity 0) via instantaneous transfers ($f_{0,j,r}$) equals the total resource capacity $C_r$.
* (4) Implicates transfer activation for instantaneous transfers ($\Delta_{i,j,r} = 0$). If a flow $f_{i,j,r}$ exists, it implies the presence of the optional interval $z_{i,j,r}$.
* (5) Flow conservation (into activity): Ensures the total resource $r$ received by activity $i$ (summed from durative transfers $z$ and instantaneous flows $f$) equals its required quantity $Q_{i,r}$.
* (6) Flow conservation (out of activity): Ensures the total resource $r$ sent from activity $i$ (summed via $z$ and $f$) equals the quantity $Q_{i,r}$ that the activity processed (or $C_r$ for the source node $i=0$).
* (7) Temporal linking: Enforces the time delay for durative transfers ($z$). It ensures a transfer $z_{i,j,r}$ starts after $a_i$ ends, and $a_j$ starts after $z_{i,j,r}$ ends.
* (8) Resource capacity (cumulative constraint): Ensures that at any time, the total resource $r$ consumed by all active activities $a_i$ ($\mathbf{Q}_{i,r}$) and all active durative transfers $z$ does not exceed the capacity $C_r$.


**Variable Definitions:**
* (9a) $a_i$: A mandatory interval variable representing the execution of activity $i$ with length $p_i$.
* (9b) $f_{i,j,r}$: An integer variable representing the amount of resource flow $r$ between $i$ and $j$, defined only for instantaneous transfers ($\Delta_{i,j,r}=0$).
* (9c) $z_{i,j,r}$: An optional interval variable representing the transfer of resource $r$ between $i$ and $j$, with length $\Delta_{i,j,r}$.


**Parameters (Input Data):**
* $\mathcal{A}$: The set of activities.
* $\mathcal{E}$: The set of precedence relations (pairs of activities).
* $\mathcal{R}$: The set of primary resources.
* $\mathcal{T}$: The set of potential transfers (triples of activity, activity, resource).
* $p_i$: The fixed processing time (duration) of activity $a_i$.
* $C_r$: The maximum capacity available for resource $r$ (vector of primary resource capacities).
* $\Delta_{i,j,r}$: The duration (delay) for a transfer $(i, j, r)$.
* $U_{i,j,r}$: The upper bound (maximum flow/capacity) of the transfer $(i, j, r)$.
* $Q_{i,r}$: The quantity of resource $r$ required by/transferred into/out of activity $i$.

### CPLEX Implementation

#### Imports

In [1]:
# from docplex.cp.model import *
# import docplex.cp.utils_visu as visu
import re
from pathlib import Path
from docplex.cp.model import CpoModel

#### Reading the data file

In [2]:
def parse_rcpsp_psplib(filepath):
    """
    Parses a .sm file (PSPLIB format for RCPSP with transfer times)
    and returns a dictionary with the project data.
    """
    with open(filepath, 'r') as f:
        content = f.read()

    data = {}
    match = re.search(r'jobs \(incl\. supersource/sink \):\s*(\d+)', content)
    data['n_jobs'] = int(match.group(1)) if match else 0
    match = re.search(r' - renewable\s*:\s*(\d+)', content)
    data['n_resources'] = int(match.group(1)) if match else 0
    n_jobs = data['n_jobs']
    n_res = data['n_resources']
    data['precedence_arcs'] = []
    prec_start = content.find('PRECEDENCE RELATIONS:')
    prec_end = content.find('****************', prec_start)
    prec_section = content[prec_start:prec_end]
    
    for line in prec_section.splitlines()[2:]:
        if not line.strip():
            continue
        parts = [int(p) for p in line.strip().split()]
        predecessor = parts[0]
        successors = parts[3:]
        for succ in successors:
            # Convert 1-based index from file to 0-based index
            data['precedence_arcs'].append((predecessor - 1, succ - 1))

    data['durations'] = []
    data['demands'] = []
    req_start = content.find('REQUESTS/DURATIONS:')
    req_end = content.find('****************', req_start)
    req_section = content[req_start:req_end]

    for line in req_section.splitlines()[3:]:
        if not line.strip():
            continue
        parts = [int(p) for p in line.strip().split()]
        data['durations'].append(parts[2])
        data['demands'].append(parts[3:]) # The rest are resource demands

    cap_start = content.find('RESOURCEAVAILABILITIES:')
    cap_end = content.find('****************', cap_start)
    cap_section = content[cap_start:cap_end]

    cap_line = cap_section.splitlines()[2]
    data['capacities'] = [int(p) for p in cap_line.strip().split()]
    data['transfer_times'] = []
    current_pos = cap_end

    for _ in range(n_res):
        tt_start = content.find('TRANSFERTIMES', current_pos)
        tt_end = content.find('****************', tt_start)
        tt_section = content[tt_start:tt_end]
        
        matrix = []
        lines = tt_section.splitlines()[3:]
        
        for i in range(n_jobs):
            line = lines[i]
            parts = [int(p) for p in line.strip().split()]
            matrix.append(parts[1:])
            
        data['transfer_times'].append(matrix)
        current_pos = tt_end
    return data

In [3]:
filename = "../data/rcpsptt/j301_a.sm"
filepath = Path(filename)
data = parse_rcpsp_psplib(filename)

A_hat = data['n_jobs'] - 1
R_hat = data['n_resources'] - 1

A_set = range(A_hat + 1) # Set of activities {0, ..., A_hat}
R_set = range(R_hat + 1) # Set of resources {0, ..., R_hat}

P = data['durations']                           # P[i]
C = data['capacities']                          # C[r]
Q = data['demands']                             # Q[i][r]
E = data['precedence_arcs']                     # List of (i, j) tuples
Delta_matrices = data['transfer_times']         # [r][i][j]

# For easier lookup, create a (i, j, r) -> delay dictionary
Delta = {
    (i, j, r): Delta_matrices[r][i][j]
    for i in A_set
    for j in A_set
    for r in R_set
}

# the upper bound U_i,j,r for any transfer is the total capacity C[r] of that resource
U = {
    (i, j, r): C[r]
    for i in A_set
    for j in A_set
    for r in R_set
    if i != j
}

# Build the T set (all potential transfers)
T = [
    (i, j, r)
    for i in A_set
    for j in A_set
    for r in R_set
    if i != j
]

# Set source demands equal to capacity
for r in range(data['n_resources']):
    data['demands'][0][r] = data['capacities'][r]

#### Create model and variables

In [4]:
mdl = CpoModel()

# (9a) Activity interval variables
a = [mdl.interval_var(name=f"a_{i}", size=P[i]) for i in A_set]

# (9b) Integer variable representing the amount of resource flow r between i and j, defined only for instantaneous transfers Delta = 0.
f = {
    (i, j, r): mdl.integer_var(min=0, max=U[i, j, r], name=f'f_{i}_{j}_{r}')
    for (i, j, r) in T
    if Delta[i, j, r] == 0
}

# (9c) Optional interval variable representing the transfer of resource 
# r between i and j, with length Delta_{i,j,r}.
z = {
    (i, j, r): mdl.interval_var(size=Delta[i, j, r], optional=True, name=f'z_{i}_{j}_{r}')
    for (i, j, r) in T
}

#### Add constraints and define objective

In [5]:
# (1) Minimize the makespan
mdl.add(mdl.minimize(mdl.end_of(a[A_hat])))

In [6]:
# (2) Precedence constraint
mdl.add([mdl.end_before_start(a[i], a[j]) for (i, j) in E])

# (3) Source capacity release
for r in R_set:
    mdl.add(mdl.sum(f[0, j, r] for j in A_set if (0, j, r) in f) == C[r])

# (4) Transfer activation
for (i, j, r) in T:
    if Delta[i, j, r] == 0:
        mdl.add(mdl.if_then(f[i, j, r] >= 1, mdl.presence_of(z[i, j, r]) == 1))

# (5) Flow conservation into activity i
for i in range(1, data['n_jobs']):
    for r in R_set:
        inflow_durative = mdl.sum(
            mdl.height_at_start(z[j, i, r], mdl.pulse(z[j, i, r], (0, U[j, i, r])))
            for j in A_set
            if Delta[j, i, r] > 0 and (j, i, r) in T
        )
        inflow_instant = mdl.sum(
            f[j, i, r]
            for j in A_set
            if Delta[j, i, r] == 0 and (j, i, r) in T
        )
        mdl.add(inflow_durative + inflow_instant == Q[i][r])

# (6) Flow conservation out of activity
for i in range(0, data['n_jobs'] - 1):
    for r in R_set:
        outflow_durative = mdl.sum(
            mdl.height_at_start(z[i, j, r], mdl.pulse(z[i, j, r], (0, U[i, j, r])))
            for j in A_set
            if Delta[i, j, r] > 0 and (i, j, r) in T
        )
        outflow_instant = mdl.sum(
            f[i, j, r]
            for j in A_set
            if Delta[i, j, r] == 0 and (i, j, r) in T
        )
        mdl.add(outflow_durative + outflow_instant == Q[i][r])

# (7) Temporal linking
for (i, j, r) in T:
        mdl.add(mdl.end_before_start(a[i], z[i, j, r]))
        mdl.add(mdl.end_before_start(z[i, j, r], a[j]))

# (8) Resource capacity
for r in R_set:
    activities_consuming = mdl.sum(
        mdl.pulse(a[i], Q[i][r])
        for i in A_set
        if Q[i][r] > 0
    )
    transfers_consuming = mdl.sum(
        mdl.pulse(z[i, j, k], (0, U[i, j, k]))
        for (i, j, k) in T
        if Delta[i, j, k] > 0 and k == r
    )
    mdl.add(activities_consuming + transfers_consuming <= C[r])

#### Solve the model

In [7]:
print("Solving model...")

res = mdl.solve(TimeLimit=300)
if res:
    print("Solution:")
    res.print_solution()
else:
    print("No solution found.")

Solving model...
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 4614 variables, 8854 constraints
 ! TimeLimit            = 300
 ! Initial process time : 0.21s (0.21s extraction + 0.00s propagation)
 !  . Log search space  : 62734.6 (before), 62734.6 (after)
 !  . Memory usage      : 16.5 MB (before), 16.5 MB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0       4614                 -
 + New bound is 37
                     2284       1677    1   F   on z_4_10_1
                     1817       1971    4   F   on z_10_7_3
                     2389       1379    7   F   on z_4_10_1
                     2443       1352    9   F   on z_8_5_3
                     1518       2206   10   F   on z_19_10_3
                     1148       2369   12   F   on z_21_6_2
   

## Aditional Resources

- **Instances for RCPSPTT**
  - https://www2.informatik.uni-osnabrueck.de/kombopt/data/rcpsp/

- **For image of the problem instance see this article:** [An efficient genetic algorithm to solve the resource-constrained project scheduling problem with transfer times](https://www.sciencedirect.com/science/article/pii/S0377221717306549)

- [Problem defintion](https://drive.google.com/file/d/1Gwfgm7mcY47d0zJWLY-SD9BVqHYsUKkP/view?usp=sharing)